In [49]:
import data
import utils
import json
import torch
import torchvision
import numpy as np
import sklearn.metrics as skm
import torchvision.models.segmentation.deeplabv3 as dlv3
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from tqdm import tqdm

## Test-related functions

In [50]:
def eval(model, test_dataloader, tgt_num_classes, tgt_ignore_label=None):
    model.eval()
    average_miou = 0
    average_ious = np.zeros((tgt_num_classes,))
    with torch.no_grad():
        for (inputs, targets) in tqdm(test_dataloader, desc='Batch progress'):
            ipts = torch.autograd.Variable(inputs).cuda()
            tgts = torch.autograd.Variable(targets).cuda()
            
            preds = model(ipts)['out']
            preds = torch.argmax(preds.cpu(), dim=1)
            tgts = torch.squeeze(targets, dim=1)

            miou, ious = utils.measure_performance(predictions=preds, targets=tgts, num_classes=tgt_num_classes, ignore_label=tgt_ignore_label)
            average_ious += ious
            average_miou += miou
            
        average_ious /= len(test_dataloader)
        average_miou /= len(test_dataloader)

    return {'average_miou': np.round(average_miou, 3), 'average_ious': np.round(average_ious, 3)}

## Test configuration loading

In [ ]:
import pprint
args = {}
with open('G:/My Drive/Master IVA/Master Thesis/Configs/eval-config.json') as json_file:
    args = json.load(json_file)
pprint.pprint(args)


## Data loading and preprocessing

In [54]:
test_dataset = data.HybridDataset(
        root_path=f'G:/My Drive/Master IVA/Master Thesis/Datasets/{args["data_source"]}/test',
        input_dir='rgb',
        target_dir='semantic_segmentation_mapped',
        ipt_transform=torchvision.transforms.Compose([torchvision.transforms.Resize((args["image_height"], args["image_width"])),
                                                      torchvision.transforms.Normalize(mean=args["train_mean"], std=args["train_std"])]),
        tgt_transform=torchvision.transforms.Compose([torchvision.transforms.Resize((args["image_height"], args["image_width"]))]),                        
        type=args['data_source'],
        labels_mapping=None)
test_dataset = Subset(test_dataset, indices=range(args['test_subset_size']))
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=args['test_batch_size'],
    shuffle=False
)

## Model under test preparation

In [ ]:
# File parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = dlv3.deeplabv3_resnet50(pretrained=False, progress=True, num_classes=len(data.SemanticLabelMapper.ID_TO_STRING['common'].keys()))
model.load_state_dict(torch.load(f'{args["model_load_folder_path"]}/deeplabv3_model.pt'))
model.to(device)
print('Loaded model to device.')

In [ ]:
test_metrics = eval(model=model, 
                    test_dataloader=test_dataloader, 
                    tgt_num_classes=len(data.SemanticLabelMapper.ID_TO_STRING['common'].keys()),
                    tgt_ignore_label=None)
test_metrics['average_ious'] = {k:v for k,v in zip(data.SemanticLabelMapper.ID_TO_STRING['common'].values(),
                                                   test_metrics['average_ious'])}
print(test_metrics)